In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.models import Model

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
model = tf.keras.models.load_model('/home/drevital/cs_video_processor/models/suzuyo')

In [ ]:
img_height, img_width = 200, 200
data_dir = '../input/obs-det-diffs/obstacle_detection_diffs'
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
    validation_split=0.3,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary',
    subset='validation')

In [ ]:
n_val_images = 7253

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('No Obstacles Detected (True Negatives): ', cm[0][0])
  print('No Obstacles Incorrectly Detected (False Positives): ', cm[0][1])
  print('Obstacles Missed (False Negatives): ', cm[1][0])
  print('Obstacles Detected (True Positives): ', cm[1][1])
  print('Total Obstacles: ', np.sum(cm[1]))

In [ ]:
metrics = model.evaluate_generator(
    val_generator,
    verbose=1)

In [ ]:
for name, value in zip(model.metrics_names, metrics):
  print(name, ': ', value)
print()


In [ ]:
labels = np.array([0] * (n_val_images//2) + [1] * (n_val_images//2))

In [ ]:
predictions = model.predict_generator(val_generator, 227, verbose=1)

In [ ]:
predictions

In [ ]:
len(labels)

In [ ]:
len(predictions)

In [ ]:
plot_cm(labels, predictions[:-1]) # Default: threshold = 0.5
plot_cm(labels, predictions[:-1], p=0.25)
plot_cm(labels, predictions[:-1], p=0.75)